In [2]:
import twoSampleTest as tst
import os
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import hurdat
import gc
import torch
import torch.nn as nn
import copy

In [3]:
print('Initializing...')
BASIN         = 'EP'
EVENT         = 'RW'
MIN_TEST_DATE = '20130101'
LAGTIME       = 24 # hours
LEADTIME      = 0 # hours
WINDTHRESH    = 35 # knots
METAPATH = 'data/quad/'
METASUF = '_TCdata.csv'
RADIALPATH = 'data/quad/'
RADIALSUF = '_rad.csv'

Initializing...


In [4]:
print('Importing hurdat metadata...')
storms = hurdat.Hurdat()
storms.storms = storms.storms.loc[storms.storms.DATE.str.slice(0, 4).astype('int') >= 1979]
storms.identify_events(25)

Importing hurdat metadata...


/home/azureuser/1d-simulations/hurdat.py:78: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  delimiter=',\s+'


In [5]:
print("Training Markov chain...")
chaintrain = storms.storms.loc[storms.storms.ID.str.slice(4, 8).astype('int') < int(MIN_TEST_DATE[0:4])].copy()
chaintrain.loc[:,'Y'] = chaintrain[EVENT].astype('int')
chaintrain = chaintrain.dropna(subset=['Y'])
chaintrain = chaintrain.loc[chaintrain.ID.str.startswith(BASIN)]
chain = tst.TCchain(chaintrain.Y.astype(np.int), chaintrain.ID, chaintrain.DATETIME, k=8, hurdat=storms)

Training Markov chain...


/home/azureuser/1d-simulations/twoSampleTest.py:233: RuntimeWarning: invalid value encountered in long_scalars
  to1[ii] = self.A[2*ii+1] / (self.A[2*ii]+self.A[2*ii+1])


In [6]:
chaintrain

,DATE,TIME,CATEGORY,LAT,LON,WIND,ID,NAME,DATETIME,RI,RW,Y
60143,19790531,1800,TD,11.0,-95.5,25,EP011979,ANDRES,1979-05-31 18:00:00,False,False,0
60144,19790601,0000,TD,11.0,-96.2,30,EP011979,ANDRES,1979-06-01 00:00:00,False,False,0
60145,19790601,0600,TD,11.0,-96.9,30,EP011979,ANDRES,1979-06-01 06:00:00,False,False,0
60146,19790601,1200,TD,11.3,-97.6,30,EP011979,ANDRES,1979-06-01 12:00:00,False,False,0
60147,19790601,1800,TD,11.8,-98.0,30,EP011979,ANDRES,1979-06-01 18:00:00,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76716,20121104,0600,LO,12.4,-121.3,25,EP172012,ROSA,2012-11-04 06:00:00,False,False,0
76717,20121104,1200,LO,12.4,-121.3,25,EP172012,ROSA,2012-11-04 12:00:00,False,False,0
76718,20121104,1800,LO,12.5,-121.4,25,EP172012,ROSA,2012-11-04 18:00:00,False,False,0
76719,20121105,0000,LO,12.8,-121.6,25,EP172012,ROSA,2012-11-05 00:00:00,False,False,0


In [7]:
chaintrain.Y.__len__()

15274

In [8]:
files = os.listdir(METAPATH)
ids = [file[0:8] for file in files if file.endswith(METASUF)]

dfs = []
tc_files = [(METAPATH+id+METASUF) for id in ids]
for ii in range(len(tc_files)):
    storm_id = ids[ii]
    df = pd.read_csv(tc_files[ii], header=0)
    df.rename(columns={'TIMESTAMP': 'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
    # interpolates latitude and wind speed between snapshots
    # snapshots not every 30 mins -> interpolate needed
    colstointerp = df.columns[~df.columns.isin(['timestamp', 'ID', 'NAME'])]
    df = df.resample('0.5H', on='timestamp').mean().reset_index()
    df[colstointerp] = df[colstointerp].interpolate()
    # round latitude to tenths place - used to compute storm image area
    df['LAT'] = df['LAT'].round(1)
    df['ID'] = storm_id
    dfs.append(df[['ID', 'timestamp', 'LAT', 'WIND']])

meta = pd.concat(dfs)
del dfs

In [9]:
print('Loading radial profiles...')
dfs = []
tc_files = [(RADIALPATH+id+RADIALSUF) for id in ids]
for ii in range(len(ids)):
    storm_id = ids[ii]
    df = pd.read_csv(tc_files[ii], header=0, skiprows=[0, 2])
    df.rename(columns={'radius': 'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
    df['ID'] = storm_id
    df.rename(columns=dict([(str(float(i)), str(i)) for i in range(5, 600 + 5, 5)]), inplace=True)
    df.sort_values('timestamp', inplace=True)
    df.reset_index(drop=True, inplace=True)
    dfs.append(df)
data = pd.concat(dfs)
data.dropna(axis=0, how='any', inplace=True)
del dfs

data = data.merge(meta, how='inner', on=['timestamp', 'ID'], suffixes=('', '_tc'))
radcols = [str(i) for i in range(5, 400 + 5, 5)]
data = data.loc[data['ID'].str.startswith(BASIN), :]

dfs = []
for group in storms.storms.ID.unique():
    tmp = storms.storms.loc[storms.storms.ID == group, ['DATETIME', 'ID', EVENT]]
    tmp = tmp.resample('0.5H', on='DATETIME').mean().reset_index()
    tmp[EVENT] = tmp[EVENT].interpolate()
    tmp['Y'] = np.floor(tmp[EVENT]).astype('int')
    tmp['ID'] = group
    tmp = tmp[['DATETIME', 'Y', 'ID']]
    dfs.append(tmp)

events = pd.concat(dfs)
del dfs
data = data.merge(events, how='left', left_on=['timestamp', 'ID'],
                                      right_on=['DATETIME', 'ID'])

Loading radial profiles...


In [10]:
print("Processing radial profile sequences...")
# slices list will be used to build full dataset
imgs = []
vectors = []
total = 0
for storm_id in data.ID.unique():
    # sorted dataframe with just single storm
    storm_df = data.loc[data['ID'] == storm_id, :].sort_values('timestamp')

    # create numpy matrices and vectors of desired values
    rad_mtx = storm_df[radcols].values

    seq_len = LAGTIME*2 + LEADTIME*2
    startidx = None
    endidx = rad_mtx.shape[0] - seq_len
    switch = False
    storm_imgs = []
    storm_vectors = []
    # iterate over all 60 row images
    for jj in range(rad_mtx.shape[0] - seq_len):
        # Extract metadata for the "anchor point"
        meta = storm_df.iloc[jj + LAGTIME].loc[['timestamp', 'ID', 'WIND', 'Y']].values
        # Only keep between first time above threshold and last time below
        if meta[2] >= WINDTHRESH:
            switch = True
            if startidx is None:
                startidx = jj
        if meta[2] < WINDTHRESH and switch:
            endidx = jj
            switch = False

        total += 1 # increment on kept slice
        # create "slices" of 60 rows
        s_rad = rad_mtx[jj:(jj + seq_len), ]

        storm_imgs.append(s_rad)   # element has seq_len rows
        storm_vectors.append(meta)
    if startidx is None:
        continue
    imgs = imgs + storm_imgs[startidx:endidx]
    vectors = vectors + storm_vectors[startidx:endidx]
x = np.stack(imgs, axis=2).transpose((2,0,1))
del imgs
z = pd.DataFrame(np.stack(vectors, axis=1).transpose(),
                 columns=['timestamp', 'ID', 'WIND', 'Y'])
del vectors
z['idx'] = z.index

Processing radial profile sequences...


In [11]:
x.shape

(64366, 48, 80)

In [12]:
xmean = np.mean(x, axis=(0,1))
x = np.subtract(x, xmean)

In [13]:
xvar = np.var(x, axis=(0,1,2))
x = np.divide(x, np.sqrt(xvar))

In [14]:
[id for id in z.ID.unique() if z.loc[z.ID==id,:].isnull().values.any()]

[]

In [15]:
print("Intializing test...")
# z = z.iloc[[ii for ii in range(len(z.ID)) if z.ID.iloc[ii,] not in ['AL082003', 'AL142003', 'AL152000', 'AL162012']],]

basinTest = tst.test2sample(train=z.loc[z['timestamp'] < MIN_TEST_DATE, :],
                            test=z.loc[z['timestamp'] >= MIN_TEST_DATE, :])

Intializing test...


In [16]:
basinTest.test_data.ID.unique().__len__()

152

# Define Model

In [17]:
class RadNet(nn.Module):
    def __init__(self: int = 1000) -> None:
        super(RadNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Try removing
            nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.regression = nn.Sequential(
            nn.Dropout(),
            nn.Linear(308, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2),
            nn.Sigmoid(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.regression(x)
        return x

In [18]:
from torchsummary import summary
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
summary(RadNet().to(device), (1, 48, 80))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 50, 82]              80
              ReLU-2            [-1, 8, 50, 82]               0
         MaxPool2d-3            [-1, 8, 25, 41]               0
            Conv2d-4            [-1, 8, 27, 43]             584
              ReLU-5            [-1, 8, 27, 43]               0
         MaxPool2d-6            [-1, 8, 13, 21]               0
            Conv2d-7            [-1, 4, 15, 23]             292
              ReLU-8            [-1, 4, 15, 23]               0
         MaxPool2d-9             [-1, 4, 7, 11]               0
          Dropout-10                  [-1, 308]               0
           Linear-11                  [-1, 256]          79,104
             ReLU-12                  [-1, 256]               0
           Linear-13                    [-1, 2]             514
          Sigmoid-14                   

# Define dataset

In [ ]:
class RadDataset(torch.utils.data.Dataset):
    def __init__(self, rad, label_vec):
        self.image = torch.unsqueeze(rad, axis=1)
        self.y = label_vec

    def __getitem__(self, index):
        return self.image[index,], self.y[index]

    def __len__(self):
        return self.image.shape[0]

# Define regressor

In [ ]:
class cnn_regressor:
    def __init__(self, variables=['idx'], BATCH_SIZE=512, N_EPOCHS=10000, EPOCHS_CHECK=10,
                 LR=1e-4, LR_DECAY=0.05, PATIENCE_EARLY_STOPPING=20, test_fraction=.2):
        # Variable here should be index in the tensor
        self.regression = RadNet()
        self.variables = variables
        self.BATCH_SIZE = BATCH_SIZE
        self.N_EPOCHS = N_EPOCHS
        self.EPOCHS_CHECK = EPOCHS_CHECK
        self.LR = LR
        self.LR_DECAY = LR_DECAY
        self.PATIENCE_EARLY_STOPPING = PATIENCE_EARLY_STOPPING
        self.test_fraction = test_fraction
        self.first_fit = True
        self.initial_fit = None
    
    def train_radnet(self, Y_train, Y_test, freeze_conv=False):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
        test_prevelance = np.mean(Y_test.astype(np.float))
        train_prevalence = np.mean(Y_train.astype(np.float))

        # Draw from x by index
        train_dataset = RadDataset(
            rad=torch.from_numpy(x[self.train_idx,].astype(np.float64)).type(torch.Tensor),
            label_vec=torch.from_numpy(Y_train).type(torch.LongTensor))
        test_dataset = RadDataset(
            rad=torch.from_numpy(x[self.test_idx,].astype(np.float64)).type(torch.Tensor),
            label_vec=torch.from_numpy(Y_test).type(torch.LongTensor))

        train_load = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=self.BATCH_SIZE, shuffle=False)
        test_load = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=self.BATCH_SIZE, shuffle=False)

        model = self.regression
        
        # freeze convolutional layers if requested
        if freeze_conv:
            print('Freezing convolutional layers...')
            for child in self.regression.children():
                if isinstance(child, nn.Conv2d):
                    for param in child.parameters():
                        param.requires_grad = False
                if isinstance(child, nn.Linear):
                    layer.reset_parameters()

        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=self.LR)
        avg = Y_train.mean()
        weights = [1/(1-avg), 1/avg]
        class_weights = torch.FloatTensor(weights).cuda()
        loss_function = nn.CrossEntropyLoss(weight=class_weights)

        # Creating a scheduler in case the LR decay is passed. 
        # In case the LR decay is 0, then the decay is set to happen
        # after the last epoch (so it's equivalent to not happening)
        step_size = self.LR_DECAY if self.LR_DECAY > 0 else self.N_EPOCHS + 1
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

        flnm_model = 'cnn_tmp'

        training_loss, test_loss, check_loss = [], [], []
        patience_counter = 0
        best_median = 1e20
        for epoch in range(self.N_EPOCHS):

            if patience_counter > self.PATIENCE_EARLY_STOPPING:
                break

            model.train()
            train_loss_temp = []
            prevs_temp = []
            for batch_idx, (image_batch, y_batch) in enumerate(train_load):
                image_batch, y_batch = image_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                out_batch = model(image_batch)
                loss = loss_function(out_batch, y_batch)
                loss.backward()
                optimizer.step()
                train_loss_temp.append(loss.item())
                prevs_temp.append(np.average(out_batch[:,1].cpu().detach().numpy()))

            scheduler.step()
            weights_average = np.ones(len(train_loss_temp))
            weights_average[-1] = (train_dataset.image.shape[0] % self.BATCH_SIZE) / self.BATCH_SIZE
            training_loss.append(np.average(train_loss_temp, weights=weights_average))
            prevs = np.average(prevs_temp, weights=weights_average)

            model.eval()
            image_batch_test, y_batch_test = next(iter(test_load))
            test_acc = y_batch_test
            image_batch_test, y_batch_test = image_batch_test.to(device), y_batch_test.to(device)
            out_batch_test = model(image_batch_test)
            test_loss.append(loss_function(out_batch_test, y_batch_test).item())

            if epoch == self.EPOCHS_CHECK:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'training_loss': training_loss,
                    'test_loss': test_loss
                }, flnm_model)

            if epoch % self.EPOCHS_CHECK == 0:
                print('Epoch: %d, Train Loss: %.4f,'
                      ' Test Loss: %.4f, Predicted Train Fraction: %.2f,'
                      ' True Train Fraction: %.2f,\nPredicted Test Fraction: %.2f,' 
                      ' True Test Fraction: %.2f'% (epoch, training_loss[-1], test_loss[-1],
                                                    prevs, train_prevalence,
                                                    torch.mean(out_batch_test[:,1]), test_prevelance))

                if epoch > (2 * self.EPOCHS_CHECK) and np.median(
                        test_loss[-self.EPOCHS_CHECK:]) > best_median:
                    patience_counter += 1
                else:
                    patience_counter = 0
                    best_median = np.median(test_loss[-self.EPOCHS_CHECK:])
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'training_loss': training_loss,
                        'test_loss': test_loss
                    }, flnm_model)
    
    def state_load(self):       
        checkpoint = torch.load('cnn_tmp')
        self.regression.load_state_dict(checkpoint['model_state_dict'])

        
        
    def fit(self, data):
        if self.first_fit:
            idx = data[self.variables]
            ids = data['ID'].unique()
            test_ids = np.random.choice(ids, size=int(ids.__len__()*self.test_fraction))
            self.test_idx = data.loc[data.ID.isin(test_ids),'idx'].values
            train_idx = np.array([idx for idx in data[self.variables].values if idx not in self.test_idx])
            self.train_idx = np.squeeze(train_idx, axis=1)
        else:
            self.regression = copy.deepcopy(self.initial_fit)
        
        Y_train = [data.Y.values[ii] for ii in range(data.shape[0]) if data[self.variables].iloc[ii].values in self.train_idx]
        Y_train = np.array(Y_train).astype(np.long)
        Y_test = [data.Y.values[ii] for ii in range(data.shape[0]) if data[self.variables].iloc[ii].values in self.test_idx]
        Y_test = np.array(Y_test).astype(np.long)
        
        self.train_radnet(Y_train, Y_test, freeze_conv=not self.first_fit)
        self.state_load()
        if self.first_fit:
            self.initial_fit = copy.deepcopy(self.regression)
            self.first_fit = False

        
    def predict(self, data):
        new_dataset = RadDataset(
            rad=torch.from_numpy(x[data.idx,].astype(np.float64)).type(torch.Tensor),
            label_vec=torch.from_numpy(data.Y.values.astype(int)).type(torch.LongTensor))
        
        new_loader = torch.utils.data.DataLoader(dataset=new_dataset, batch_size=data.shape[0], shuffle=False)
        x_new, y_new = next(iter(new_loader))
        y_pred = self.regression(x_new.to(device)).cpu().detach().numpy()
        y_pred = np.divide(y_pred, np.expand_dims(np.sum(y_pred, axis=1), axis=1))
        
        return y_pred[:,1]

In [ ]:
reg = cnn_regressor(EPOCHS_CHECK=20, PATIENCE_EARLY_STOPPING=10, 
                    LR=1e-5, LR_DECAY=0.01, test_fraction=.4)
reg.fit(basinTest.train_data)

In [ ]:
y_pred = reg.predict(basinTest.test_data)
y_valid = basinTest.test_data.Y.values.astype(int)

In [ ]:
valid_dataset = RadDataset(
            rad=torch.from_numpy(x[basinTest.test_data.idx,].astype(np.float64)).type(torch.Tensor),
            label_vec=torch.from_numpy(basinTest.test_data.Y.values.astype(int)).type(torch.LongTensor))
valid_load = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=17969, shuffle=False)
x_valid, y_valid = next(iter(valid_load))
y_valid = y_valid.detach().numpy()
y_pred = reg.predict(basinTest.test_data)

In [ ]:
pstar = 0.5
tp = sum([(y_pred[ii] > pstar) for ii in range(y_valid.shape[0]) if y_valid[ii] == 1])
fp = sum([(y_pred[ii] > pstar) for ii in range(y_valid.shape[0]) if y_valid[ii] != 1])
tn = sum([(y_pred[ii] < pstar) for ii in range(y_valid.shape[0]) if y_valid[ii] != 1])
fn = sum([(y_pred[ii] < pstar) for ii in range(y_valid.shape[0]) if y_valid[ii] == 1])
tpr = tp/(tp+fn)
tnr = tn/(tn+fp)

In [ ]:
tpr, tnr, (tpr+tnr)/2

In [ ]:
testreg = cnn_regressor(N_EPOCHS=1000, EPOCHS_CHECK=20, PATIENCE_EARLY_STOPPING=10, 
                        LR=2e-5, LR_DECAY=0.01, test_fraction=.4)
basinTest.test(chain, testreg, pb=True, groupvar='ID', B=100)

basinTest.test_data.to_csv('results/ENP-RW-TestData-update.csv')
basinTest.train_data.to_csv('results/ENP-RW-TrainData-update.csv')
np.savetxt('results/ENP-RW-NullDsn-functional-update.csv', basinTest.P)

In [ ]:
basinTest.get_global()